# Model Deployment using Keras

## 1. Introduction
In this workbook, we will train a simple Keras MNIST CNN model and deploy that for inference

Parts of this workbook are borrowed from [here](https://keras.io/examples/vision/mnist_convnet/)

## 2. Imports and Dependencies.
The few packages needed are loaded next. Particularly, `numpy`, `tensorflow`, `keras`, `mlflow` will be majorly used in this tutorial. `requests` package will be used for performing query. `json` is used to post and get response from the server.

In [1]:
import os
import sys
import mlflow
import mlflow.keras
import numpy as np
from mlflow import pyfunc
import cloudpickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from mlflow.utils.environment import _mlflow_conda_env

# Setting a tracking uri to log the mlflow logs in a particular location tracked by 
from mlflow.tracking import MlflowClient
tracking_uri = os.environ.get("TRACKING_URL")
client = MlflowClient(tracking_uri=tracking_uri)
mlflow.set_tracking_uri(tracking_uri)

2021-11-12 18:35:43.301898: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-12 18:35:43.301935: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## 3.Training

In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11501568/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1600)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

2021-11-12 18:35:45.382578: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-12 18:35:45.382613: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-12 18:35:45.382637: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ov9ogvb8-58556b7474-d8g7w): /proc/driver/nvidia/version does not exist
2021-11-12 18:35:45.382942: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
mlflow.keras.log_model(model, artifact_path="keras-model")

2021-11-12 18:35:50.743054: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/15
422/422 [==============================] - 17s 40ms/step - loss: 0.3598 - accuracy: 0.8932 - val_loss: 0.0867 - val_accuracy: 0.9765
Epoch 2/15
422/422 [==============================] - 16s 38ms/step - loss: 0.1151 - accuracy: 0.9647 - val_loss: 0.0577 - val_accuracy: 0.9855
Epoch 3/15
422/422 [==============================] - 16s 39ms/step - loss: 0.0871 - accuracy: 0.9738 - val_loss: 0.0507 - val_accuracy: 0.9867
Epoch 4/15
422/422 [==============================] - 16s 38ms/step - loss: 0.0743 - accuracy: 0.9767 - val_loss: 0.0433 - val_accuracy: 0.9883
Epoch 5/15
422/422 [==============================] - 16s 38ms/step - loss: 0.0627 - accuracy: 0.9809 - val_loss: 0.0427 - val_accuracy: 0.9877
Epoch 6/15
422/422 [==============================] - 16s 38ms/step - loss: 0.0592 - accuracy: 0.9820 - val_loss: 0.0348 - val_accuracy: 0.9908
Epoch 7/15
422/422 [==============================] - 16s 37ms/step - loss: 0.0536 - accuracy: 0.9838 - val_loss: 0.0354 - val_accuracy:

2021-11-12 18:39:54.085673: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmpwm3_2595/model/data/model/assets


# 3. Some utility functions

In [5]:
from mlflow.utils.environment import _mlflow_conda_env
import tensorflow as tf
import cloudpickle

conda_env = _mlflow_conda_env(
    additional_conda_deps=[
        "tensorflow=={}".format(tf.__version__),
    ],
    additional_pip_deps=[
        "cloudpickle=={}".format(cloudpickle.__version__),
        "mlflow=={}".format(mlflow.__version__),
    ])

class KerasMnistCNN(mlflow.pyfunc.PythonModel):

    def load_context(self, context):
        import tensorflow as tf
        self.graph = tf.Graph()
        with self.graph.as_default():
            K.set_learning_phase(0)
            self.model = mlflow.keras.load_model(context.artifacts["keras-model"])

    def predict(self, context, input_df):
        with self.graph.as_default():
            return self.model.predict(input_df.values.reshape(-1, 28, 28, 1))

mlflow.pyfunc.log_model(
    artifact_path="keras-pyfunc",
    python_model=KerasMnistCNN(),
    artifacts={
        "keras-model": mlflow.get_artifact_uri("keras-model")
    },
    conda_env=conda_env)

## 4. Deploying the model
The above code logs a model in the experiments tab. For more info please refer [here](https://rocketml.gitbook.io/rocketml-user-guide/experiments). After deploying the model, we can obtain the model url for performing query as shown below.

## 5. Query from the server

There are two methods to perform query... The first is using `requests` library and the other using `curl` shell command.

In [7]:
import requests
import json

url = "http://127.0.0.1:5011/invocations"
headers = {"Content-Type":"text/csv"}

# First case, run inference on single data point
np_array = np.random.rand(1,6).tolist()
json_data = json.dumps(np_array)
response = requests.post(url,data=json_data,headers=headers)
if response.status_code == 200:
    output = np.array(json.loads(response.json())).astype(np.float32)
    print(output)
else:
    print(response.status_code)
    print("REST API deployment is in progress -- please try again in a few minutes!")

# Second case, run inference on multiple data points
np_array = np.random.rand(20,6).tolist()
json_data = json.dumps(np_array)
response = requests.post(url,data=json_data,headers=headers)
if response.status_code == 200:
    output = np.array(json.loads(response.json())).astype(np.float32)
    print(output)
else:
    print(response.status_code)
    print("REST API deployment is in progress -- please try again in a few minutes!")

400
REST API deployment is in progress -- please try again in a few minutes!
400
REST API deployment is in progress -- please try again in a few minutes!


In [8]:
!curl http://127.0.0.1:5011/invocations -H 'Content-Type:text/csv' -d '[[0.6499166977064089, 0.17579454262114602, 0.2688911143313131, 0.7146591854799202, 0.6497433572112488, 0.7723469203958951]]'

{"error_code": "BAD_REQUEST", "message": "Encountered an unexpected error while evaluating the model. Verify that the serialized input Dataframe is compatible with the model for inference.", "stack_trace": "Traceback (most recent call last):\n  File \"/anaconda/envs/mlflow-0eb65c492db9b60cb18b7df13009a4ce7ffc117b/lib/python3.7/site-packages/mlflow/pyfunc/scoring_server/__init__.py\", line 303, in transformation\n    raw_predictions = model.predict(data)\n  File \"/anaconda/envs/mlflow-0eb65c492db9b60cb18b7df13009a4ce7ffc117b/lib/python3.7/site-packages/mlflow/pyfunc/__init__.py\", line 608, in predict\n    return self._model_impl.predict(data)\n  File \"/anaconda/envs/mlflow-0eb65c492db9b60cb18b7df13009a4ce7ffc117b/lib/python3.7/site-packages/mlflow/keras.py\", line 473, in predict\n    predicted = _predict(data)\n  File \"/anaconda/envs/mlflow-0eb65c492db9b60cb18b7df13009a4ce7ffc117b/lib/python3.7/site-packages/mlflow/keras.py\", line 460, in _predict\n    predicted = pd.DataFrame(sel

In [9]:
print("Traceback (most recent call last):\n  File \"/anaconda/envs/mlflow-0eb65c492db9b60cb18b7df13009a4ce7ffc117b/lib/python3.7/site-packages/mlflow/pyfunc/scoring_server/__init__.py\", line 303, in transformation\n    raw_predictions = model.predict(data)\n  File \"/anaconda/envs/mlflow-0eb65c492db9b60cb18b7df13009a4ce7ffc117b/lib/python3.7/site-packages/mlflow/pyfunc/__init__.py\", line 608, in predict\n    return self._model_impl.predict(data)\n  File \"/anaconda/envs/mlflow-0eb65c492db9b60cb18b7df13009a4ce7ffc117b/lib/python3.7/site-packages/mlflow/keras.py\", line 473, in predict\n    predicted = _predict(data)\n  File \"/anaconda/envs/mlflow-0eb65c492db9b60cb18b7df13009a4ce7ffc117b/lib/python3.7/site-packages/mlflow/keras.py\", line 460, in _predict\n    predicted = pd.DataFrame(self.keras_model.predict(data.values))\n  File \"/anaconda/envs/mlflow-0eb65c492db9b60cb18b7df13009a4ce7ffc117b/lib/python3.7/site-packages/keras/utils/traceback_utils.py\", line 67, in error_handler\n    raise e.with_traceback(filtered_tb) from None\n  File \"/anaconda/envs/mlflow-0eb65c492db9b60cb18b7df13009a4ce7ffc117b/lib/python3.7/site-packages/keras/engine/training.py\", line 1804, in predict\n    raise ValueError('Unexpected result of `predict_function` '\nValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.\n")

Traceback (most recent call last):
  File "/anaconda/envs/mlflow-0eb65c492db9b60cb18b7df13009a4ce7ffc117b/lib/python3.7/site-packages/mlflow/pyfunc/scoring_server/__init__.py", line 303, in transformation
    raw_predictions = model.predict(data)
  File "/anaconda/envs/mlflow-0eb65c492db9b60cb18b7df13009a4ce7ffc117b/lib/python3.7/site-packages/mlflow/pyfunc/__init__.py", line 608, in predict
    return self._model_impl.predict(data)
  File "/anaconda/envs/mlflow-0eb65c492db9b60cb18b7df13009a4ce7ffc117b/lib/python3.7/site-packages/mlflow/keras.py", line 473, in predict
    predicted = _predict(data)
  File "/anaconda/envs/mlflow-0eb65c492db9b60cb18b7df13009a4ce7ffc117b/lib/python3.7/site-packages/mlflow/keras.py", line 460, in _predict
    predicted = pd.DataFrame(self.keras_model.predict(data.values))
  File "/anaconda/envs/mlflow-0eb65c492db9b60cb18b7df13009a4ce7ffc117b/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
    raise e.with_traceback(fi